In [17]:
import streamlit as st
import pandas as pd
import numpy as np
from model import ModelClinicValue
from model import ModelTransformTransactionData
import pickle
import os


In [18]:
file_name = 'clinic_value_set.pkl'

# Initialize or load existing dictionary
if os.path.exists(file_name):
    # Load existing data
    with open(file_name, 'rb') as f:
        output_variables_set = pickle.load(f)
else:
    # Start with an empty dictionary
    output_variables_set = {}

In [19]:
with open(r'dummy_clinic_model/pkl_files/dataset_1/pool_clinic_df.pkl', 'rb') as f:
    pool_clinic_df = pickle.load(f)

uploaded_file_set = list(pool_clinic_df.keys())


In [20]:


base_ebit = 250000
base_equipment_value = 80826.5
base_net_sales_growth = 0.1
base_number_of_active_patients = 1000
base_number_of_dentist = 2
base_relative_variability_patient_spending = 0.25
base_risk_of_leaving_dentist = 0
base_fitout_cost = 1728000000 / 10000 # 5 times of Indonesia cost converted into AUD
base_ebit_ratio = 0.22
base_relative_variability_net_sales = 0.15
base_equipment_usage_ratio = 0.5
base_projected_number_of_dentist = 2
base_last_fitout = 5

        
base_ebit_multiple = 2.5


In [23]:



# # read pkl file
# with open(r'dummy_clinic_model/pkl_files/dataset_1/pool_clinic_df.pkl', 'rb') as f:
#     pool_clinic_df = pickle.load(f)

# uploaded_file = list(pool_clinic_df.keys())[0]


for uploaded_file in uploaded_file_set:

    # Initialize the ModelClinicValue class
    model = ModelClinicValue({})

    transaction_df = pool_clinic_df[uploaded_file]['Gross Profit']
    indirect_cost_df = pool_clinic_df[uploaded_file]['Indirect Cost']
    model_transform = ModelTransformTransactionData(transaction_df, indirect_cost_df)
    transformed_uploaded_file = model_transform.prepare_all_data()


    # If a file is uploaded, update the model variables with data from the file
    if uploaded_file:
        model.update_variable_from_uploaded_dictionary(transformed_uploaded_file)
        # Previously were
        # model.update_variable_from_uploaded_file(transformed_uploaded_file)





    # Dictionary to store the variables
    company_variables = {}


    net_sales = float(model.net_sales) if model.net_sales is not None else 0.0
    company_variables["Net Sales"] = net_sales

    cogs = float(model.cogs) if model.cogs is not None else 0.0
    company_variables["COGS"] = cogs
        
        
    trading_income = value=float(model.trading_income) if model.trading_income is not None else 0.0
    company_variables["Trading Income"] = trading_income
            
    net_sales_growth = float(model.net_sales_growth) if model.net_sales_growth is not None else 0.0

    relative_variability_net_sales = float(model.relative_variability_net_sales) if model.relative_variability_net_sales is not None else 0.0


    other_income = float(model.other_income) if model.other_income is not None else 0.0
    company_variables["Other Income"] = other_income

    interest_revenue = float(model.interest_revenue) if model.interest_revenue is not None else 0.0
    company_variables["Interest Revenue of Bank"] = interest_revenue

    operational_expense = float(model.operational_expense) if model.operational_expense is not None else 0.0
    company_variables["Operational Expense"] = operational_expense

    other_expense = value=float(model.other_expense) if model.other_expense is not None else 0.0
    company_variables["Other Expense"] = other_expense


    clinic_depreciation = float(model.clinic_depreciation) if model.clinic_depreciation is not None else 0.0
    company_variables["Depreciation of Equipment Clinic Expense"] = clinic_depreciation


    non_clinic_depreciation = float(model.non_clinic_depreciation) if model.non_clinic_depreciation is not None else 0.0
    company_variables["Depreciation of Equipment Non Clinic Expense"] = non_clinic_depreciation


    bank_tax_expense = float(model.bank_tax_expense) if model.bank_tax_expense is not None else 0.0
    company_variables["Bank Tax Expense"] = bank_tax_expense

    other_tax = float(model.other_tax) if model.other_tax is not None else 0.0
    company_variables["Other Tax"] = other_tax

    # Create a DataFrame with the new columns
    data_df = pd.read_csv('equipment_data.csv')

    # Display the DataFrame using st.data_editor with modified column configurations
    company_variables['Equipment_Life'] = st.data_editor(
        data_df,
        column_config={
            "Equipment": st.column_config.SelectboxColumn(
                "Equipment",
                help="Select the equipment or write your own value",
                options=[
            "Intra Oral Camera",
            "Bleaching Unit",
            "Ultrasonic Scaler",
            "Light Cure",
            "Dental Unit",
            "Portable Xrays",
            "Endomotor",
            "Autoclaves",
            "Ultrasonic Cleaner",
            "Water Tank",
            "Prophylaxis Hand Piece",
            "Handpiece Set",
            "Compressor",
            "Apex Locator",
            "Dental Loupe",
            "Portable Light",
            "Camera DSLR",
            "Water Tank Hose",
            "Sealing Machine",
            "Xray Sensor"
        ],
                required=True,
            ),
            "Own?": st.column_config.CheckboxColumn(
                "Own?",
                help="Does your clinic own the specific equipment?",
            ),
            "Expected Lifetime": st.column_config.NumberColumn(
                "Expected Lifetime",
                help="Expected operational lifetime of the equipment in years", disabled=True
            ),
            "Current Lifetime Usage": st.column_config.NumberColumn(
                "Current Lifetime Usage",
                help="The number of years the equipment has been in use (on average if multiple units)",
            ),
        },
        hide_index=True,
        use_container_width=True,
        num_rows='dynamic',
        column_order=["Equipment", "Own?", "Expected Lifetime", "Current Lifetime Usage"]
    )
    
    selected_equipment = company_variables['Equipment_Life'][company_variables['Equipment_Life']['Own?'] == True]
    company_variables["Equipments Value"] = (selected_equipment['Quantity'] * selected_equipment['Price']).sum()
        
        
        

            
    fitout_value = base_fitout_cost if model.fitout_value == 0 else model.fitout_value
    company_variables["Fitout Value"] = fitout_value

    last_fitout = int(base_last_fitout if model.last_fitout_year == 0 else model.last_fitout_year)
    company_variables['Last Fitout Year'] = last_fitout



    if model.dentist_contribution is None:
        dentist_availability = pd.read_csv('dentist_contribution_tofill.csv')
    else:
        dentist_availability = model.dentist_contribution
        


        
        
        
    dentist_availability = st.data_editor(dentist_availability, num_rows='dynamic')

        

        
    number_of_active_patients = int(model.number_of_patients) if model.number_of_patients is not None else 0
            

    relative_variability_patient_spending = float(model.relative_variability_patient_spending) if model.relative_variability_patient_spending is not None else 0.0
        




    model.equipment_life = selected_equipment
    company_variables['Equipment Life'] = model.equipment_life

    # equipment_usage_ratio, total_equipments, total_remaining_value = model.calculate_equipment_usage_ratio()
            

                
    # risk = model.calculate_risk_of_dentist_leaving(dentist_availability)
                
            
    # df_risk = dentist_availability[dentist_availability['Possibly Leaving?'] == True][['Dentist Provider', 'Sales Contribution ($)']]
                
        
        # Create the output_variables dictionary
    output_variables = {
            'Net Sales': company_variables.get('Net Sales', 0),
            'COGS': company_variables.get('COGS', 0),
            'Trading Income': company_variables.get('Trading Income', 0),
            'Other Income': company_variables.get('Other Income', 0),
            'Interest Revenue of Bank': company_variables.get('Interest Revenue of Bank', 0),
            'Advertising & Promotion Expense': company_variables.get('Advertising & Promotion Expense', 0),
            'Operational Expense': company_variables.get('Operational Expense', 0),
            'Other Expense': company_variables.get('Other Expense', 0),
            'Depreciation of Equipment Clinic Expense': company_variables.get('Depreciation of Equipment Clinic Expense', 0),
            'Depreciation of Equipment Non Clinic Expense': company_variables.get('Depreciation of Equipment Non Clinic Expense', 0),
            'Bank Tax Expense': company_variables.get('Bank Tax Expense', 0),
            'Other Tax': company_variables.get('Other Tax', 0),
            'Equipments Value': company_variables.get('Equipments Value', 0),
            # 'Net Cash Flow Average': average_cashflow,
            # 'Net Cash Flow Standard Deviation': std_deviation,
            # 'Net Cash Flow Trend Coefficient': trend_coefficient,
            # 'Equipment Usage Ratio': equipment_usage_ratio,
            # 'Total Equipments': total_equipments,
            'Net Sales Growth': net_sales_growth,
            # 'Total Remaining Value': total_remaining_value,
            # 'Current Number of Dentist': number_of_dentist,
            # 'Projected Number of Dentist': projected_number_of_dentist,
            # 'Possibility Existing Dentist Leaving': possibility_existing_dentist_leaving,
            'Relative Variation of Net Sales': relative_variability_net_sales,
            'Number of Active Patients': number_of_active_patients,
            'Relative Variation of Patient Spending': relative_variability_patient_spending,
            # 'Risk of Leaving Dentist': risk,
            'Fitout Value': company_variables.get("Fitout Value", 0),
            'Last Fitout Year': company_variables.get("Last Fitout Year", 0),
            'Equipment Life': company_variables.get('Equipment Life', None)
            
        }

    output_variables['EBIT'] = model.ebit
    output_variables['EBIT Ratio'] = model.ebit_ratio
    output_variables['General Expense'] = output_variables['Operational Expense'] + output_variables['Other Expense'] + output_variables['Advertising & Promotion Expense']
    output_variables['Patient Pool'] = transaction_df['Patient ID'].unique().tolist()
        
        # output_variables['Operating Income'] = output_variables['Net Sales'] + output_variables['Trading Income'] + output_variables['Other Income'] + output_variables['Interest Revenue of Bank'] + output_variables['COGS'] + output_variables['Advertising & Promotion Expense'] + output_variables['Operational Expense'] + output_variables['Other Expense']

        # # Convert the output_variables dictionary to a DataFrame and save as CSV
        # output_df = pd.DataFrame([output_variables])
        # output_df.to_csv('output_variables.csv', index=False)
        

        

    ebit_multiple = model.ebit_baseline_to_multiple(output_variables['Net Sales Growth'])

    # equipment_adjusting_value = model.equipment_adjusting_value(output_variables['Total Remaining Value'], base_equipment_value, base_equipment_usage_ratio)
    fitout_adjusting_value = model.fitout_adjusting_value(output_variables['Fitout Value'], output_variables['Last Fitout Year'], base_fitout_cost, base_last_fitout)

    # ebit_multiple = model.ebit_multiple_adjustment_due_dentist(ebit_multiple, output_variables['Risk of Leaving Dentist'])
    ebit_multiple = model.ebit_multiple_adjustment_due_net_sales_variation(ebit_multiple, output_variables['Relative Variation of Net Sales'])

    ebit_multiple = model.ebit_multiple_adjustment_due_number_patient_and_patient_spending_variability(ebit_multiple, output_variables['Number of Active Patients'], output_variables['Relative Variation of Patient Spending'])

    clinic_valuation = ebit_multiple * model.ebit
        

    clinic_valuation_adjusted = clinic_valuation + fitout_adjusting_value
    output_variables['EBIT Multiple'] = ebit_multiple
    output_variables['Clinic Valuation Adjusted'] = clinic_valuation_adjusted
    output_variables['Clinic Valuation'] = clinic_valuation





        
    ID = uploaded_file
        
        



    # Update or add the new data
    output_variables_set[ID] = output_variables


        

    
  

In [22]:
len(output_variables_set)

50

In [24]:
# Save the updated dictionary back to the file
with open(file_name, 'wb') as f:
    pickle.dump(output_variables_set, f)
        